## Week 4, Lab 1: Predicting Left-Handedness from Psychological Factors
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

We'll walk through a full data science problem in this lab. 

In [19]:
# Import libraries 
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

---
## Step 1: Define The Problem.

You're currently a data scientist working at a university. A professor of psychology is attempting to study the relationship between personalities and left-handedness. They have tasked you with gathering evidence so that they may publish.

Specifically, the professor says "I need to prove that left-handedness is caused by some personality trait. Go find that personality trait and the data to back it up."

As a data scientist, you know that any real data science problem must be **specific** and **conclusively answerable**. For example:
- Bad data science problem: "What is the link between obesity and blood pressure?"
    - This is vague and is not conclusively answerable. That is, two people might look at the conclusion and one may say "Sure, the problem has been answered!" and the other may say "The problem has not yet been answered."
- Good data science problem: "Does an association exist between obesity and blood pressure?"
    - This is more specific and is conclusively answerable. The problem specifically is asking for a "Yes" or "No" answer. Based on that, two independent people should both be able to say either "Yes, the problem has been answered" or "No, the problem has not yet been answered."
- Excellent data science problem: "As obesity increases, how does blood pressure change?"
    - This is very specific and is conclusively answerable. The problem specifically seeks to understand the effect of one variable on the other.

### 1. In the context of the left-handedness and personality example, what are three specific and conclusively answerable problems that you could answer using data science? 

> You might find it helpful to check out the codebook in the repo for some inspiration.

Answer: 
1. Are left-handed people more likely to play video games compared to right-handed people?
2. Are left-handed people more likely to like horoscope compared to right-handed people? 
3. Are left-handed people more likely to dance when they are alone compared to right-handed people?

---
## Step 2: Obtain the data.

### 2. Read in the file titled "data.csv."
> Hint: Despite being saved as a .csv file, you won't be able to simply `pd.read_csv()` this data!

In [2]:
df = pd.read_csv('data.csv', sep = '\t')
df.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,country,fromgoogle,engnat,age,education,gender,orientation,race,religion,hand
0,4,1,5,1,5,1,5,1,4,1,...,US,2,1,22,3,1,1,3,2,3
1,1,5,1,4,2,5,5,4,1,5,...,CA,2,1,14,1,2,2,6,1,1
2,1,2,1,1,5,4,3,2,1,4,...,NL,2,2,30,4,1,1,1,1,2
3,1,4,1,5,1,4,5,4,3,5,...,US,2,1,18,2,2,5,3,2,2
4,5,1,5,1,5,1,5,1,3,1,...,US,2,1,22,3,1,1,3,2,3


### 3. Suppose that, instead of us giving you this data in a file, you were actually conducting a survey to gather this data yourself. From an ethics/privacy point of view, what are three things you might consider when attempting to gather this data?
> When working with sensitive data like sexual orientation or gender identity, we need to consider how this data could be used if it fell into the wrong hands!

Answer:
1. Use ambigious column headers and store the data dictionary somewhere else
2. Ensure that sensitive data are masked (E.g. Name -> ID0001,ID0002) 
3. Group the data if possible (E.g. Age: <18 y/o, 18-25 y/o etc)

---
## Step 3: Explore the data.

### 4. Conduct exploratory data analysis on this dataset.
> If you haven't already, be sure to check out the codebook in the repo, as that will help in your EDA process.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4184 entries, 0 to 4183
Data columns (total 56 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Q1           4184 non-null   int64 
 1   Q2           4184 non-null   int64 
 2   Q3           4184 non-null   int64 
 3   Q4           4184 non-null   int64 
 4   Q5           4184 non-null   int64 
 5   Q6           4184 non-null   int64 
 6   Q7           4184 non-null   int64 
 7   Q8           4184 non-null   int64 
 8   Q9           4184 non-null   int64 
 9   Q10          4184 non-null   int64 
 10  Q11          4184 non-null   int64 
 11  Q12          4184 non-null   int64 
 12  Q13          4184 non-null   int64 
 13  Q14          4184 non-null   int64 
 14  Q15          4184 non-null   int64 
 15  Q16          4184 non-null   int64 
 16  Q17          4184 non-null   int64 
 17  Q18          4184 non-null   int64 
 18  Q19          4184 non-null   int64 
 19  Q20          4184 non-null 

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Q1,4184.0,1.962715,1.360291,0.0,1.0,1.0,3.00,5.0
Q2,4184.0,3.829589,1.551683,0.0,3.0,5.0,5.00,5.0
Q3,4184.0,2.846558,1.664804,0.0,1.0,3.0,5.00,5.0
Q4,4184.0,3.186902,1.476879,0.0,2.0,3.0,5.00,5.0
Q5,4184.0,2.865440,1.545798,0.0,1.0,3.0,4.00,5.0
Q6,4184.0,3.672084,1.342238,0.0,3.0,4.0,5.00,5.0
Q7,4184.0,3.216539,1.490733,0.0,2.0,3.0,5.00,5.0
Q8,4184.0,3.184512,1.387382,0.0,2.0,3.0,4.00,5.0
Q9,4184.0,2.761233,1.511805,0.0,1.0,3.0,4.00,5.0
Q10,4184.0,3.522945,1.242890,0.0,3.0,4.0,5.00,5.0


Majority of the columns should have 1 as the min value for it to be a valid response. This means that there are some invalid responses in Q1-Q44, engnat, education, gender, orientation, race, religion and hand.

---
## Step 4: Model the data.

### 5. Suppose I wanted to use Q1 - Q44 to predict whether or not the person is left-handed. Would this be a classification or regression problem? Why?

Answer: Classification. The output of the model would be categorical - 'Left-handed'/2 or 'Right-handed'/1

### 6. We want to use $k$-nearest neighbors to predict whether or not a person is left-handed based on their responses to Q1 - Q44. Before doing that, however, you remember that it is often a good idea to standardize your variables. In general, why would we standardize our variables? Give an example of when we would standardize our variables.

Answer: We standardise our variables when the scales for different variables varies greatly. Standardisation will make all variables contribution equally. For e.g. we would need to standardise our variables if we have 2 variables in different scales - income (in the thousands) and age (in the tens).

### 7. Give an example of when we might not standardize our variables.

Answer: This case is such an example. Our variables (Q1-Q44) are of the same scale (0-5). 

### 8. Based on your answers to 6 and 7, do you think we should standardize our predictor variables in this case? Why or why not?

Answer: No. Q1-Q44 have the same scale.

### 9. We want to use $k$-nearest neighbors to predict whether or not a person is left-handed. What munging/cleaning do we need to do to our $y$ variable in order to explicitly answer this question? Do it.

Answer: Remove responses that are neither left nor right handed

In [5]:
y = df[df['hand'].isin([1,2])]['hand']
y.value_counts()

1    3542
2     452
Name: hand, dtype: int64

### 10. The professor for whom you work suggests that you set $k = 4$. In this specific case, why might this be a bad idea?

Answer: There are some people who are ambidexterous. If we have an equal number, it would be difficult to determine if he/she is left or right handed.

### 11. Let's *(finally)* use $k$-nearest neighbors to predict whether or not a person is left-handed!

> Be sure to create a train/test split with your data!

> Create four separate models, one with $k = 3$, one with $k = 5$, one with $k = 15$, and one with $k = 25$.

> Instantiate and fit your models.

In [6]:
df.columns

Index(['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11',
       'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21',
       'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27', 'Q28', 'Q29', 'Q30', 'Q31',
       'Q32', 'Q33', 'Q34', 'Q35', 'Q36', 'Q37', 'Q38', 'Q39', 'Q40', 'Q41',
       'Q42', 'Q43', 'Q44', 'introelapse', 'testelapse', 'country',
       'fromgoogle', 'engnat', 'age', 'education', 'gender', 'orientation',
       'race', 'religion', 'hand'],
      dtype='object')

In [7]:
x = df[df['hand'].isin([1,2])]

In [8]:
x = x.drop(columns = ['introelapse', 'testelapse', 'country',
       'fromgoogle', 'engnat', 'age', 'education', 'gender', 'orientation',
       'race', 'religion', 'hand'])

In [9]:
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [10]:
k = [3,5,15,25]
for i in k: 
    print ('k=' + str(i) + ':')
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(X_train,y_train)
    print('Train score: ', round(knn.score(X_train,y_train),5))
    print('Test score: ', round(knn.score(X_test,y_test),5), '\n')

k=3:
Train score:  0.90266
Test score:  0.84481 

k=5:
Train score:  0.89171
Test score:  0.86483 

k=15:
Train score:  0.88638
Test score:  0.88736 

k=25:
Train score:  0.88638
Test score:  0.88861 



Being good data scientists, we know that we might not run just one type of model. We might run many different models and see which is best.

### 12. We want to use logistic regression to predict whether or not a person is left-handed. Before we do that, let's check the [documentation for logistic regression in sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). Is there default regularization? If so, what is it? If not, how do you know?

Answer: L2 penalty term (Ridge regularisation) is applied by default

### 13. We want to use logistic regression to predict whether or not a person is left-handed. Before we do that, should we standardize our features?

Answer: No, the features are of the same scale

### 14. Let's use logistic regression to predict whether or not the person is left-handed.


> Be sure to use the same train/test split with your data as with your $k$-NN model above!

> Create four separate models, one with LASSO and $\alpha = 1$, one with LASSO and $\alpha = 10$, one with Ridge and $\alpha = 1$, and one with Ridge and $\alpha = 10$. *(Hint: Be careful with how you specify $\alpha$ in your model!)*

> Instantiate and fit your models.

In [11]:
c = [1,1/10]
penalty = ['l1','l2']
for i in c:
    for j in penalty: 
        if j == 'l1': 
            print ('LASSO, alpha=' + str(1/i) + ':')
        else: 
            print ('Ridge, alpha=' + str(1/i) + ':')
        lr = LogisticRegression(penalty = j, C = i, solver = 'liblinear')
        lr.fit(X_train,y_train)
        print('Train score: ', round(lr.score(X_train,y_train),5))
        print('Test score: ', round(lr.score(X_test,y_test),5), '\n')

LASSO, alpha=1.0:
Train score:  0.88638
Test score:  0.88861 

Ridge, alpha=1.0:
Train score:  0.88638
Test score:  0.88861 

LASSO, alpha=10.0:
Train score:  0.88638
Test score:  0.88986 

Ridge, alpha=10.0:
Train score:  0.88638
Test score:  0.88986 



---
## Step 5: Evaluate the model(s).

### 15. Before calculating any score on your data, take a step back. Think about your $X$ variable and your $Y$ variable. Do you think your $X$ variables will do a good job of predicting your $Y$ variable? Why or why not? What impact do you think this will have on your scores?

Answer: No. Everyone has a different opinion on what a scale of 1 to 5 mean for each question.

### 16. Using accuracy as your metric, evaluate all eight of your models on both the training and testing sets. Put your scores below. (If you want to be fancy and generate a table in Markdown, there's a [Markdown table generator site linked here](https://www.tablesgenerator.com/markdown_tables#).)
- Note: Your answers here might look a little weird. You didn't do anything wrong; that's to be expected!

Answer: 
    
k=3 - 
Train score:  0.90266
Test score:  0.84481 

k=5 -
Train score:  0.89171
Test score:  0.86483 

k=15 -
Train score:  0.88638
Test score:  0.88736 

k=25 -
Train score:  0.88638
Test score:  0.88861 
    
LASSO, alpha=1.0 -
Train score:  0.88638
Test score:  0.88861 

Ridge, alpha=1.0 -
Train score:  0.88638
Test score:  0.88861 

LASSO, alpha=10.0 -
Train score:  0.88638
Test score:  0.88986 

Ridge, alpha=10.0 -
Train score:  0.88638
Test score:  0.88986     

### 17. In which of your $k$-NN models is there evidence of overfitting? How do you know?

Answer: k = 3 and 5. Train score is more than Test score

### 18. Broadly speaking, how does the value of $k$ in $k$-NN affect the bias-variance tradeoff? (i.e. As $k$ increases, how are bias and variance affected?)

Answer: As k increases, bias increases and variance decreases

### 19. If you have a $k$-NN model that has evidence of overfitting, what are three things you might try to do to combat overfitting?

Answer:
1. Increase k 
2. Get more data (in terms of rows) 
3. Reduce model complexity by removing irrelavant features

### 20. In which of your logistic regression models is there evidence of overfitting? How do you know?

Answer: None. The train and test scores are similar

### 21. Broadly speaking, how does the value of $C$ in logistic regression affect the bias-variance tradeoff? (i.e. As $C$ increases, how are bias and variance affected?)

Answer: As C increases, bias decreases and variance increases

### 22. For your logistic regression models, play around with the regularization hyperparameter, $C$. As you vary $C$, what happens to the fit and coefficients in the model? What do you think this means in the context of this specific problem?

Answer: Scores and coefficients are not affected much. 

### 23. If you have a logistic regression model that has evidence of overfitting, what are three things you might try to do to combat overfitting?

Answer:

1. Reduce C (increase regularisation strength)
2. Get more data (in terms of rows) 
3. Reduce model complexity by removing irrelavant features

---
## Step 6: Answer the problem.

### 24. Suppose you want to understand which psychological features are most important in determining left-handedness. Would you rather use $k$-NN or logistic regression? Why?

Answer: Logistic Regression. The coefficients allow us to determine the importance

### 25. Select your logistic regression model that utilized LASSO regularization with $\alpha = 1$. Interpret the coefficient for `Q1`.

In [12]:
lr = LogisticRegression(penalty = 'l1', C = 1, solver = 'liblinear')
lr.fit(X_train,y_train)
lr.coef_

array([[-0.01179536, -0.03940118, -0.01247924, -0.08905016,  0.04776221,
         0.01711782,  0.03336763, -0.13579756, -0.07455161,  0.08551908,
         0.02712159, -0.03547005, -0.02670564,  0.0191807 , -0.00516023,
         0.02313761,  0.02839055,  0.00929114, -0.04720512, -0.03467391,
        -0.06205486, -0.09674804, -0.05485878,  0.00235589,  0.06399587,
         0.10042934,  0.01881793,  0.01170107,  0.06649328,  0.04350763,
         0.02632677, -0.04751038, -0.03045617, -0.02390211,  0.02160716,
         0.        , -0.02645912,  0.0852944 , -0.07772408, -0.06854754,
        -0.04937054, -0.04363031, -0.10630035,  0.00423631]])

Answer: For e.g. when the score of Q1 increases by 1, it means that left-handedness is e-0.01186463 = 0.988 times as likely

### 26. If you have to select one model overall to be your *best* model, which model would you select? Why?
- Usually in the "real world," you'll fit many types of models but ultimately need to pick only one! (For example, a client may not understand what it means to have multiple models, or if you're using an algorithm to make a decision, it's probably pretty challenging to use two or more algorithms simultaneously.) It's not always an easy choice, but you'll have to make it soon enough. Pick a model and defend why you picked this model!

Answer: Logistic Regression. It has a similar train and test score and runs faster than KNN

### 27. Circle back to the three specific and conclusively answerable questions you came up with in Q1. Answer one of these for the professor based on the model you selected!

1. Are left-handed people more likely to play video games compared to right-handed people? (Q37)
2. Are left-handed people more likely to like horoscope compared to right-handed people?
3. Are left-handed people more likely to dance when they are alone compared to right-handed people?

In [17]:
lr.coef_[0][36]

-0.026459116361322

In [20]:
np.exp(lr.coef_[0][36])

0.9738878591009774

Answer: Left-handed people are more likely to play video games compared to right-handed people

### BONUS:
Looking for more to do? Probably not - you're busy! But if you want to, consider exploring the following. (They could make for a blog post!)
- Create a visual plot comparing training and test metrics for various values of $k$ and various regularization schemes in logistic regression.
- Rather than just evaluating models based on accuracy, consider using sensitivity, specificity, etc.
- In the context of predicting left-handedness, why are unbalanced classes concerning? If you were to re-do this process given those concerns, what changes might you make?
- Fit and evaluate a generalized linear model other than logistic regression (e.g. Poisson regression).
- Suppose this data were in a `SQL` database named `data` and a table named `inventory`. What `SQL` query would return the count of people who were right-handed, left-handed, both, or missing with their class labels of 1, 2, 3, and 0, respectively? (You can assume you've already logged into the database.)